In [1]:
'''
IMPORTS
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import torch
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import os
from tqdm import tqdm

import os
import time
import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
import torch.nn.functional as F
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import torch
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils import data
import itertools
import re
import random
import time
from torch.autograd import Variable
import math
from scipy.ndimage import gaussian_filter
from torch.autograd import Variable
from tqdm import tqdm
import json

import os
import torch
import dill
from e2cnn import gspaces
# from e2cnn import nn
import torch.nn as nn
import numpy as np
from torch.utils import data

import warnings
warnings.filterwarnings('ignore')

In [2]:
disease_dict = {}

for plant_disease in os.listdir('Dataset/'):
    disease_dict[plant_disease] = len(os.listdir('Dataset/' + plant_disease))

disease_dict

{'Apple___Apple_scab': 1000,
 'Apple___Black_rot': 1000,
 'Apple___Cedar_apple_rust': 1000,
 'Apple___healthy': 1645,
 'Background_without_leaves': 1143,
 'Blueberry___healthy': 1502,
 'Cherry___healthy': 1000,
 'Cherry___Powdery_mildew': 1052,
 'Corn___Cercospora_leaf_spot Gray_leaf_spot': 1000,
 'Corn___Common_rust': 1192,
 'Corn___healthy': 1162,
 'Corn___Northern_Leaf_Blight': 1000,
 'Grape___Black_rot': 1180,
 'Grape___Esca_(Black_Measles)': 1383,
 'Grape___healthy': 1000,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 1076,
 'Orange___Haunglongbing_(Citrus_greening)': 5507,
 'Peach___Bacterial_spot': 2297,
 'Peach___healthy': 1000,
 'Pepper,_bell___Bacterial_spot': 1000,
 'Pepper,_bell___healthy': 1478,
 'Potato___Early_blight': 1000,
 'Potato___healthy': 1000,
 'Potato___Late_blight': 1000,
 'Raspberry___healthy': 1000,
 'Soybean___healthy': 5090,
 'Squash___Powdery_mildew': 1835,
 'Strawberry___healthy': 1000,
 'Strawberry___Leaf_scorch': 1109,
 'Tomato___Bacterial_spot': 2127,

In [3]:
def data_loader(path, batch_size):
    
    transform = transforms.Compose(
        [transforms.Resize(255), transforms.CenterCrop(224), transforms.ToTensor()]
    )
    
    dataset = datasets.ImageFolder(path, transform = transform)
    
    indices = list(range(len(dataset)))
    
    split = int(np.floor(0.85 * len(dataset)))
    
    validation = int(np.floor(0.70 * split))
    
    np.random.shuffle(indices)
    
    train_indices, validation_indices, test_indices = (
        indices[:validation//16],
        indices[validation//16:split//16],
        indices[split//16:((split//16)+100)]
    )
    
    train_sampler = SubsetRandomSampler(train_indices)
    validation_sampler = SubsetRandomSampler(validation_indices)
    test_sampler = SubsetRandomSampler(test_indices)
    
    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=train_sampler
    )
    test_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=test_sampler
    )
    validation_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, sampler=validation_sampler
    )    
    
    return dataset, train_loader, validation_loader, test_loader

In [4]:
dataset, train_loader, validation_loader, test_loader = data_loader(path = 'Dataset/', batch_size = 16)

In [5]:
len(train_loader)

143

In [6]:
classes = len(dataset.class_to_idx)

In [7]:
device = "cpu"

In [8]:
def batch_gd(model, criterion, train_loader, validation_loader, test_laoder, epochs):
    train_losses = np.zeros(epochs)
    validation_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)

    for e in range(epochs):
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in tqdm(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            output = model(inputs)

            loss = criterion(output, targets)

            train_loss.append(loss.item())  # torch to numpy world

            loss.backward()
            optimizer.step()

        train_loss = np.mean(train_loss)

        validation_loss = []

        for inputs, targets in tqdm(validation_loader):

            inputs, targets = inputs.to(device), targets.to(device)

            output = model(inputs)

            loss = criterion(output, targets)

            validation_loss.append(loss.item())  # torch to numpy world

        validation_loss = np.mean(validation_loss)

        train_losses[e] = train_loss
        validation_losses[e] = validation_loss

        dt = datetime.now() - t0

        print(
            f"Epoch : {e+1}/{epochs} Train_loss:{train_loss:.3f} Test_loss:{validation_loss:.3f} Duration:{dt}"
        )

    return train_losses, validation_losses

In [11]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.fc = nn.Linear(2048, 39)
model.eval()

Using cache found in C:\Users\DELL/.cache\torch\hub\pytorch_vision_v0.10.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
torch.cuda.set_per_process_memory_fraction(0.8)

train_losses, validation_losses = batch_gd(
    model, criterion, train_loader, validation_loader, 60
)